In [319]:
import pandas as pd
import numpy as np
from sympy.combinatorics import Permutation
import subprocess

p = 'santa-2023/'
path = pd.read_csv(p + 'puzzles.csv')
info = pd.read_csv(p + 'puzzle_info.csv')
sub = pd.read_csv(p+'sample_submission.csv')

info['allowed_moves_count'] = info['allowed_moves'].map(lambda x: {k: Permutation(v) for k, v in eval(x).items()})
paths = pd.merge(path, info, how='left', on='puzzle_type')
paths = pd.merge(paths, sub, how='left', on='id')

In [ ]:
# https://github.com/dwalton76/rubiks-cube-NxNxN-solver/tree/master

# git clone https://github.com/dwalton76/rubiks-cube-NxNxN-solver.git
# cd rubiks-cube-NxNxN-solver
# make init
# pip install kociemba

In [455]:
# N=2
# N=3
# N=4
# N=5
# N=6
# N=7
# N=8
# N=9
# N=10
# N=19
N=33

NN=N*N

In [456]:
# org = "ABCDEF"
# import itertools
# org
# p = itertools.permutations(org)
# for a in p:
#     print(a)

In [457]:
# "ACBDEF"
# "URFDLB"
# s="ABCDEF"
# s[0] + s[2] + s[1] + s[5] + s[4] + s[3]
dct=dict(zip("ACBFED", "URFDLB"))
dct

{'A': 'U', 'C': 'R', 'B': 'F', 'F': 'D', 'E': 'L', 'D': 'B'}

In [458]:
def state2ubl(state):

    # dct = {}
    # for i,u in enumerate(U):
    #     dct[state.split(';')[4+i*NN]] = u
    # print(dct)
    s = ''.join([dct[f] for f in state.split(';')])
    
    return s[:NN] + s[2*NN:3*NN] + s[NN:2*NN] + s[5*NN:] + s[4*NN:5*NN] + s[3*NN:4*NN]

In [459]:
eval(info.loc[info['puzzle_type'] == f'cube_{N}/{N}/{N}']['allowed_moves'].values[0]).keys()

dict_keys(['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'r0', 'r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9', 'r10', 'r11', 'r12', 'r13', 'r14', 'r15', 'r16', 'r17', 'r18', 'r19', 'r20', 'r21', 'r22', 'r23', 'r24', 'r25', 'r26', 'r27', 'r28', 'r29', 'r30', 'r31', 'r32', 'd0', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8', 'd9', 'd10', 'd11', 'd12', 'd13', 'd14', 'd15', 'd16', 'd17', 'd18', 'd19', 'd20', 'd21', 'd22', 'd23', 'd24', 'd25', 'd26', 'd27', 'd28', 'd29', 'd30', 'd31', 'd32'])

In [460]:
moves = eval(info.loc[info['puzzle_type'] == f'cube_{N}/{N}/{N}']['allowed_moves'].values[0])
for move in list(moves):
    moves['-'+move] = np.argsort(moves[move]).tolist()

# TODO: 本質的に使わない操作が存在する？
# 他を回すと自分を固定する
# https://github.com/dwalton76/rubiks-cube-NxNxN-solver/tree/master/rubikscubennnsolver
M = {}
M["U"] = f'-d{N-1}'
M["R"] = "r0"
M["B"] = f"-f{N-1}"
M["F"] = "f0"
M["L"] = f"-r{N-1}"
M["D"] = "d0"
# if N>=4:
M["Uw"] = f'-d{N-1}.-d{N-2}'
M["Rw"] = "r0.r1"
M["Bw"] = f"-f{N-1}.-f{N-2}"
M["Fw"] = "f0.f1"
M["Lw"] = f"-r{N-1}.-r{N-2}"
M["Dw"] = "d0.d1"
# if N>=6:
#     M["2Uw"] = f'-d{N-1}.-d{N-2}'
#     M["2Rw"] = "r0.r1"
#     M["2Bw"] = f"-f{N-1}.-f{N-2}"
#     M["2Fw"] = "f0.f1"
#     M["2Lw"] = f"-r{N-1}.-r{N-2}"
#     M["2Dw"] = "d0.d1"
#     M["3Uw"] = f'-d{N-1}.-d{N-2}.-d{N-3}'
#     M["3Rw"] = "r0.r1.r2"
#     M["3Bw"] = f"-f{N-1}.-f{N-2}.-f{N-3}"
#     M["3Fw"] = "f0.f1.f2"
#     M["3Lw"] = f"-r{N-1}.-r{N-2}.-r{N-3}"
#     M["3Dw"] = "d0.d1.d2"
for i in range(2, N):
    M[f"{i}Uw"] = ".".join([f"-d{N-1-k}" for k in range(i)])
    M[f"{i}Rw"] = ".".join([f"r{k}" for k in range(i)])
    M[f"{i}Bw"] = ".".join([f"-f{N-1-k}" for k in range(i)])
    M[f"{i}Fw"] = ".".join([f"f{k}" for k in range(i)])
    M[f"{i}Lw"] = ".".join([f"-r{N-1-k}" for k in range(i)])
    M[f"{i}Dw"] = ".".join([f"d{k}" for k in range(i)])
    
for m in list(M):
    M[m+"2"] = M[m] + '.' + M[m]
    if "-" in M[m]:
        M[m+"'"] = M[m].replace("-","")
    else:
        # M[m+"'"] = "-"+M[m]
        M[m+"'"] = "-"+".-".join(M[m].split("."))
M

{'U': '-d32',
 'R': 'r0',
 'B': '-f32',
 'F': 'f0',
 'L': '-r32',
 'D': 'd0',
 'Uw': '-d32.-d31',
 'Rw': 'r0.r1',
 'Bw': '-f32.-f31',
 'Fw': 'f0.f1',
 'Lw': '-r32.-r31',
 'Dw': 'd0.d1',
 '2Uw': '-d32.-d31',
 '2Rw': 'r0.r1',
 '2Bw': '-f32.-f31',
 '2Fw': 'f0.f1',
 '2Lw': '-r32.-r31',
 '2Dw': 'd0.d1',
 '3Uw': '-d32.-d31.-d30',
 '3Rw': 'r0.r1.r2',
 '3Bw': '-f32.-f31.-f30',
 '3Fw': 'f0.f1.f2',
 '3Lw': '-r32.-r31.-r30',
 '3Dw': 'd0.d1.d2',
 '4Uw': '-d32.-d31.-d30.-d29',
 '4Rw': 'r0.r1.r2.r3',
 '4Bw': '-f32.-f31.-f30.-f29',
 '4Fw': 'f0.f1.f2.f3',
 '4Lw': '-r32.-r31.-r30.-r29',
 '4Dw': 'd0.d1.d2.d3',
 '5Uw': '-d32.-d31.-d30.-d29.-d28',
 '5Rw': 'r0.r1.r2.r3.r4',
 '5Bw': '-f32.-f31.-f30.-f29.-f28',
 '5Fw': 'f0.f1.f2.f3.f4',
 '5Lw': '-r32.-r31.-r30.-r29.-r28',
 '5Dw': 'd0.d1.d2.d3.d4',
 '6Uw': '-d32.-d31.-d30.-d29.-d28.-d27',
 '6Rw': 'r0.r1.r2.r3.r4.r5',
 '6Bw': '-f32.-f31.-f30.-f29.-f28.-f27',
 '6Fw': 'f0.f1.f2.f3.f4.f5',
 '6Lw': '-r32.-r31.-r30.-r29.-r28.-r27',
 '6Dw': 'd0.d1.d2.d3.d4.d5',
 '7U

In [466]:
# paths.loc[paths['puzzle_type'] == 'cube_3/3/3'].head()
# paths.loc[paths['puzzle_type'] == 'cube_4/4/4'].head()
# paths.loc[paths['puzzle_type'] == 'cube_5/5/5'].head()
# paths.loc[paths['puzzle_type'] == 'cube_6/6/6'].head()
# paths.loc[paths['puzzle_type'] == 'cube_7/7/7'].head()
# paths.loc[paths['puzzle_type'] == 'cube_33/33/33'].head()

In [462]:
i=0

row=paths.loc[paths['puzzle_type'] == f'cube_{N}/{N}/{N}'].iloc[i]
id = row['id']
cur_state = row['initial_state']
print(cur_state)
print(id, state2ubl(cur_state))

B;F;C;B;E;F;A;F;F;C;B;A;F;A;D;C;C;B;E;E;C;D;B;D;E;D;C;D;B;F;C;A;C;A;B;A;A;E;E;A;E;A;D;C;C;C;F;C;F;A;E;E;A;E;D;D;F;D;B;F;E;D;B;D;B;D;C;E;F;A;A;E;D;F;E;F;C;A;A;C;E;B;A;F;B;E;D;E;E;F;C;B;A;B;D;A;A;E;D;A;B;A;D;B;B;F;F;C;D;C;A;E;D;C;C;F;E;E;B;A;F;C;D;F;A;A;C;F;B;D;D;B;D;F;D;D;D;B;F;C;C;E;B;F;D;A;C;F;F;E;D;F;E;A;B;B;B;B;D;E;B;A;A;C;A;F;D;B;C;B;C;F;C;B;E;C;C;E;D;F;D;D;B;F;A;E;C;A;F;B;B;C;D;D;C;F;C;C;F;D;F;D;F;E;E;B;C;F;B;F;D;A;A;F;A;B;D;A;E;E;C;A;D;B;B;B;B;F;A;A;C;A;C;F;E;B;A;C;D;D;E;B;D;D;C;E;A;B;B;F;C;D;C;C;B;B;B;E;C;F;B;F;C;F;D;D;B;B;F;D;E;C;C;E;A;D;B;F;A;A;B;F;B;E;C;F;D;C;C;C;D;B;C;A;C;F;D;B;A;A;F;E;E;E;D;D;A;B;C;C;A;F;A;A;A;E;A;C;A;A;C;D;F;B;D;D;C;F;C;F;D;C;B;D;B;C;F;F;C;F;A;B;D;A;F;B;C;C;B;F;C;E;E;F;B;D;C;A;A;A;A;F;B;C;B;C;F;A;B;F;E;C;B;A;B;A;A;B;C;D;F;F;A;D;E;F;B;F;E;F;D;F;E;C;A;C;F;A;E;E;D;A;A;D;C;F;A;E;A;E;B;A;B;E;A;D;B;F;E;C;C;A;C;A;B;A;A;B;C;D;A;C;B;F;C;C;E;D;E;C;F;F;E;A;C;C;E;C;C;E;C;C;D;D;E;E;E;F;C;B;A;B;B;E;F;C;D;A;A;B;A;A;F;C;A;A;D;C;A;A;D;F;E;F;D;E;B;B;F;E;B;E;A;E;E;E;B;A;D;A;

In [450]:
result=subprocess.run(f"python ./rubiks-cube-NxNxN-solver/rubiks-cube-solver.py --state {state2ubl(cur_state)}", shell=True, capture_output=True)
output=result.stdout.decode()
if not output.startswith("Solution: "):
    print("Fail!", output, result.stderr.decode())
sol=output.replace("Solution: ","").replace("\n","")
sol

"F 9Uw2 9Rw B2 9Lw' U 9Fw 9Rw' D 9Fw' 9Rw2 8Fw' U' 9Uw' 9Fw2 9Lw2 9Dw' 8Bw 9Fw2 9Uw F 8Lw' B2 9Dw U' 8Rw' 8Fw 8Rw' F' 8Lw 8Dw 8Rw' 9Uw F' 9Lw2 9Uw2 L2 9Bw2 7Fw 7Lw 9Fw2 9Uw' 7Bw' R2 9Dw 8Fw2 8Uw' 8Bw2 7Fw' 8Rw2 8Uw F' 8Dw' 8Bw2 7Lw2 D' F 8Rw2 7Bw 7Dw' F' 7Uw2 7Lw' 7Fw' 9Rw2 6Fw 9Uw2 9Rw2 B 9Dw' R' 6Bw 9Dw 6Lw' 9Dw L' 9Uw' L' 8Uw2 6Bw 8Rw2 U' 8Uw2 6Lw 8Uw' F 8Uw' 6Rw' 8Dw2 R' 7Rw 7Bw' L2 7Lw B' 7Lw 7Bw' 7Rw' 7Rw2 7Dw2 6Lw 7Uw2 7Rw2 7Dw2 F2 7Lw2 D 6Lw' 6Rw' 7Dw 6Rw' 6Uw2 6Bw2 6Fw 6Uw 6Rw' 9Fw2 R' 9Dw' 9Lw2 R' 9Dw F 9Uw' R2 5Fw' 9Uw 8Uw 5Bw 8Rw2 8Uw2 8Fw2 8Uw R2 8Bw2 F' 8Dw' 8Fw2 8Dw 7Rw2 5Bw' 7Uw' 7Fw2 7Rw2 5Dw2 7Rw2 5Bw 7Dw' 5Lw' 7Uw2 F 7Rw2 6Rw 6Fw' 6Uw L2 6Dw' 6Rw2 6Uw2 D2 6Bw 6Rw' 5Rw 5Bw 6Dw' 6Bw2 6Lw2 6Rw2 F2 5Fw' 6Uw' 5Lw2 5Fw' 6Dw' 5Rw2 5Dw 5Bw' 5Lw' 5Rw 5Bw' 4Dw 4Fw' 9Lw2 4Rw 9Bw2 9Uw2 9Fw2 4Rw 4Fw' 4Dw 9Fw2 F' 8Dw2 8Lw2 4Rw 8Rw2 8Uw' 8Dw 4Rw' 8Fw2 8Dw' 4Lw' 8Uw' 8Dw 4Rw 7Dw2 4Bw 7Uw 7Rw2 7Fw2 U2 7Bw2 4Rw2 U2 4Bw' 7Dw R 4Bw' 4Rw' 6Dw' 6Lw2 6Uw' 4Rw 6Dw' 6Rw2 B2 4Bw 6Uw' 5Rw2 5Bw

In [451]:
# https://github.com/dwalton76/rubiks-cube-NxNxN-solver?tab=readme-ov-file
# sol="D U' B' L B' U2 R U' L F U L2 U2 R2 F2 U R2 B2 U F2"
# sol="D' L2 Bw2 Rw' D Fw' D' Rw2 U Bw2 D B L Dw2 Rw' L2 U' F D2 Dw2 Rw2 F' L2 D' Bw2 Lw2 D2 F2 R2 D' Lw2 U L D B2 L' D L B' D2 F' R2 B2 D R2 U2 R2 D2 B2 L2 D'"

In [452]:
new_state = cur_state
mmoves = '.'.join([M[m] for m in sol.split(' ')])

for move in mmoves.split('.'):
    new_state = ';'.join(list(np.asarray(new_state.split(';'))[np.array(moves[move])]))

# I = ['r0.r1.r2','d0.d1.d2','f0.f1.f2']
I = [".".join([f"r{i}"for i in range(N)]), ".".join([f"d{i}"for i in range(N)]), ".".join([f"f{i}"for i in range(N)])]
for init_moves in [''] + I + [i1 + '.' + i2 for i1 in I for i2 in I]+ [i1 + '.' + i2+ '.' + i3 for i1 in I for i2 in I for i3 in I]+ [i1 + '.' + i2+ '.' + i3 + '.' + i4 for i1 in I for i2 in I for i3 in I for i4 in I]:
    temp_state = new_state
    if len(init_moves) > 0:
        for move in init_moves.split('.'):
            temp_state = ';'.join(list(np.asarray(temp_state.split(';'))[np.array(moves[move])]))

    if temp_state == row['solution_state']:
        print(f'solved id: {id}')
        if len(init_moves) > 0:
            mmoves += '.' + init_moves
        if len(paths.iloc[id,7].split('.')) > len(mmoves.split('.')):
            print(f"improved: new length {len(mmoves.split('.'))} vs current length {len(paths.iloc[id,7].split('.'))}")
            paths.iloc[id,7] = mmoves
        break


solved id: 277
improved: new length 15067 vs current length 24419


# まとめて解く

In [463]:
import subprocess

for i, row in paths.loc[paths['puzzle_type'] == f'cube_{N}/{N}/{N}'].iterrows():
    id = row['id']
    cur_state = row['initial_state']
    problem=state2ubl(cur_state)
    print("-"*30)
    print(id, problem)

    result=subprocess.run(f"python ./rubiks-cube-NxNxN-solver/rubiks-cube-solver.py --state {problem}", shell=True, capture_output=True)
    output=result.stdout.decode()
    if not output.startswith("Solution: "):
        print("Fail!", output, result.stderr.decode())
        continue
    sol=output.replace("Solution: ","").replace("\n","")
    print("solved", sol)

    new_state = cur_state
    mmoves = '.'.join([M[m] for m in sol.split(' ')])
    
    for move in mmoves.split('.'):
        new_state = ';'.join(list(np.asarray(new_state.split(';'))[np.array(moves[move])]))
    
    I = [".".join([f"r{i}"for i in range(N)]), ".".join([f"d{i}"for i in range(N)]), ".".join([f"f{i}"for i in range(N)])]
    for init_moves in [''] + I + [i1 + '.' + i2 for i1 in I for i2 in I]+ [i1 + '.' + i2+ '.' + i3 for i1 in I for i2 in I for i3 in I]+ [i1 + '.' + i2+ '.' + i3 + '.' + i4 for i1 in I for i2 in I for i3 in I for i4 in I]:
        temp_state = new_state
        if len(init_moves) > 0:
            for move in init_moves.split('.'):
                temp_state = ';'.join(list(np.asarray(temp_state.split(';'))[np.array(moves[move])]))
    
        if temp_state == row['solution_state']:
            print(f'solved id: {id}')
            if len(init_moves) > 0:
                mmoves += '.' + init_moves
            if len(paths.iloc[id,7].split('.')) > len(mmoves.split('.')):
                print(f"improved: new length {len(mmoves.split('.'))} vs current length {len(paths.iloc[id,7].split('.'))}")
                paths.iloc[id,7] = mmoves
            break


------------------------------
281 FDRFLDUDDRFUDUBRRFLLRBFBLBRBFDRURUFUULLULUBRRRDRDULLULBBDBFDLBFBFBRLDUULBDLDRUURLFUDFLBLLDRFUFBUULBUFUBFFDDRBRULBRRDLLFUDRBDUURDFBBFBDBBBFDRRLFDBURDDLBDLUFFFFBLFUURUDBFRFRDRFLRRLBDBBFDULRUDFFRBBRDRRDBDBDLLFRDFDBUUDUFBULLRUBFFFFDUURURDLFURBBLFBBRLUFFDRBRRFFFLRDFDRDBBFFDBLRRLUBFDUUFDFLRDBRRRBFRURDBFUUDLLLBBUFRRUDUUULURUURBDFBBRDRDBRFBFRDDRDUFBUDFRRFDRLLDFBRUUUUDFRFRDUFDLRFUFUUFRBDDUBLDFDLDBDLRURDULLBUUBRDULULFUFLUBFDLRRURUFUUFRBURFDRRLBLRDDLURRLRRLRRBBLLLDRFUFFLDRBUUFUUDRUUBRUUBDLDBLFFDLFLULLLFUBULLFLUDURFFRFFFFFDDULDUDBBBDFDRBFBULFLFBBBRRBDURULFFLRRFDUFFLRLFULLULBBRFDUDRBUDLULDULLDUDBBRDLRFRURLFBLRDLLDULUBBDDUDLRFDBDLFURRURDBBRDFFUFFRDDFURULFFURBRRDDUFULBUDDBBLLDRRBDBDUBFLFBRRDFBLLLRBRBUBDFFBBRLUDRRULRLUBDULBULBRLLRUURUFDFBLLFUDDFLBLFRRRDDRFULLUFFLDDFRUFLLURLDLRUUFUBFUDDUBBFDFBRRULULDDDFBBRFUDRLBDFRLBDBBFRBRFBUBFUFUDLLUBURFUUBBUDFULBUURFULLLURULUUBFRBBURLRBRFDDLLFUBDLULRRBLDLRULUDBDBULLRDFLRDLDDRBRBFDLRUUBDUULDFDFBLFUBFFDUUDRDRFFUUFBLUDBBUUURFRLDFUUBBD

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



KeyError: 'N1056'

In [467]:
paths[["id", "moves"]].to_csv("solver_submit.csv", index=None)